In [ ]:
!pip install transformers
import pandas as pd
import os
import csv
import numpy as np 
import torch
from transformers import BertTokenizer, BertForMaskedLM
from google.colab import files
data = files.upload()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

In [ ]:
def import_function(filename):
  df = pd.read_csv(filename)
  oldName = df.columns[0]
  df = df.rename(columns={oldName:'MaskedText'})
  print(df.head())
  df = df.head(201)
  return df

input_df = import_function(filename = '/content/MaskedDataEMPTYMASK.csv' )

In [ ]:
# Guess what's behind the mask
def unmask_sentence(index, text, tokenizer, model, top_k=5):

  text = "[CLS] %s [SEP]"%text
  tokenized_text = tokenizer.tokenize(text)
  masked_index = tokenized_text.index("[MASK]")
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])

  column_names = ['Index','Sentence','1_word','1_weight','2_word','2_weight','3_word','3_weight','4_word','4_weight','5_word','5_weight']  
  df  = pd.DataFrame(columns = column_names)
  df['Sentence']= [text]
  df['Index']= [index]
  print(text)

  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0] 
  probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
  top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)
    
  for i, pred_idx in enumerate(top_k_indices):
    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
    print(predicted_token)
    print( top_k_weights[i])
    # best_guess=predicted_token
    token_weight = float(top_k_weights[i])
    if i==0:
      df['1_word']= [predicted_token]
      df['1_weight'] =[token_weight]
    if i==1:
      df['2_word']= [predicted_token]
      df['2_weight'] =[token_weight]
    if i==2:
      df['3_word']= [predicted_token]
      df['3_weight'] =[token_weight]
    if i==3:
      df['4_word']= [predicted_token]
      df['4_weight'] =[token_weight]
    if i==4:
      df['5_word']= [predicted_token]
      df['5_weight'] =[token_weight]
    
  return df
  


In [ ]:
for i, text  in enumerate(input_df['MaskedText']):
  if i<5:
   print(i)
   print(text)

In [ ]:
def unmask_sentence_desired_words(index, text, tokenizer, model, target_words, top_k=1000):
  target_words = ["he", "she", "they", "applicants", "candidates"]
  text = "[CLS] %s [SEP]"%text
  tokenized_text = tokenizer.tokenize(text)
  masked_index = tokenized_text.index("[MASK]")
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])

  column_names = ['Index','Sentence'] + target_words
  for name in column_names:
    if name != "Index" and name != "Sentence":
      name = name + "_weight"
  # df  = pd.DataFrame(columns = column_names)
  df  = pd.DataFrame(columns = ['Index','Sentence'])
  df['Sentence']= [text]
  df['Index']= [index]
  print(text)

  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]  
  probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
  top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)
    
  for i, pred_idx in enumerate(top_k_indices):
    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]  
    if predicted_token in target_words:
      token_weight = float(top_k_weights[i])                               
      df[predicted_token+"_weight"]=[token_weight]
  print(df.columns)
  return df
  


In [ ]:

def get_output_from_text(Index,enter_input_text,target_words_list):
  # Do a check to prevent punctuation predictions for final mask
  if enter_input_text.split(" ")[-1]=="[MASK]":
    enter_input_text+=" [MASK]" # preventing punctuation predictions for final mask
  
  # Create the target_word_column_names
  target_word_weight_list = []
  for name in target_words:
    target_word_weight_list.append(name + "_weight")
  # print(target_word_weight_list)
  
  # Create an empty df with all relevant columnnames
  sent_and_top5  = pd.DataFrame(columns = (['Index','Sentence']+ target_word_weight_list))
  
  # Get the mask predictions and ...
  while "[MASK]" in enter_input_text:
    sent_and_top5_add=unmask_sentence_desired_words(Index,enter_input_text, tokenizer, model, target_words=target_words_list)
    sent_and_top5 = pd.concat([sent_and_top5,sent_and_top5_add])
    words=enter_input_text.split(" ")
    for i in range(len(words)):
      if words[i]=="[MASK]":
        words[i]="unmasked"
        break
    enter_input_text=' '.join(words)

  
  return sent_and_top5







In [ ]:
# input_df is now the input
# input_df is MaskedDatacsvfile
target_words = ["he", "she", "they", "applicants", "candidates"]
print(input_df.columns)
for i, text in enumerate(input_df['MaskedText']):
    if i == 0:
      output_df = get_output_from_text(i,text,target_words)
    else:
      addition = get_output_from_text(i,text,target_words)
      output_df = pd.concat([output_df,addition])

print(output_df)

In [ ]:
output_df

,Index,Sentence,he_weight,she_weight,they_weight,applicants_weight,candidates_weight
0,0,[CLS] [MASK] . [SEP],0.000256,NaN,NaN,NaN,NaN
0,1,[CLS] The role involves supporting jobseekers ...,0.000591,0.000566,0.095774,0.569606,0.117771
0,1,[CLS] The role involves supporting jobseekers ...,NaN,NaN,0.000172,0.000093,0.000310
0,2,[CLS] Be reliable. [MASK] must enjoy working ...,0.001325,0.000551,0.021603,0.002211,0.001409
0,2,[CLS] Be reliable. unmasked must enjoy workin...,0.000011,0.000006,0.000102,0.000002,0.000004
...,...,...,...,...,...,...,...
0,199,[CLS] Food is unmasked passion and passion is ...,0.000037,0.000034,0.000098,NaN,NaN
0,199,[CLS] Food is unmasked passion and passion is ...,0.000035,0.000037,0.000040,NaN,NaN
0,199,[CLS] Food is unmasked passion and passion is ...,0.005101,0.001599,0.001646,0.000065,0.000062
0,199,[CLS] Food is unmasked passion and passion is ...,0.000007,0.000006,0.000005,NaN,NaN


In [ ]:
def export_function(df, filename):
  df.to_csv(filename)

export_function(output_df, filename ='/content/MaskedDataUnmaskedList.csv') 